# Modifications de résultats des sondages

## Import préliminaires des librairies

In [1]:
import re
import numpy as np
import pandas as pd
import locale
from locale import atof

# Read files
import PyPDF2
from pdfminer.high_level import extract_text

## Extraction utilisant une lecture de PDF

In [2]:
file_path="rapport_personnalite/2021-02-05.pdf"

In [3]:

text = extract_text(file_path,page_numbers=[11])

print(text)


 

 
 
 
 
 
 
 

 

 

 

Les traits d’image associés au président de la République 

 
 
 
 
 
 
Question  : 

Pour chacune des appréciations suivantes, dites si elle correspond bien 
ou  mal  à  l’idée  que  vous  vous  faites  d’Emmanuel  Macron  comme 
président de la République. 

 
 

 

 Correspond très bien  

 

 Correspond très mal 

 

 Ne se prononce pas 

TOTAL 
Corres-
pond 
bien 

TOTAL 
Corres-
pond mal 

 

 

Défend bien les intérêts 
de la France à l’étranger 

12%

41%

27%

20%

53  %  47  % 

Renouvelle la fonction
présidentielle

Mène une bonne
politique économique

A une vision pour l’avenir 
des Français 

Est proche des
préoccupations des
Français

9%

32%

32%

27%

41  %  59  % 

6%

34%

34%

26%

40  %  60  % 

9%

29%

35%

27%

38  %  62  % 

5%

28%

34%

33%

33  %  67  % 

 

Ifop-Fiducial pour Paris Match et Sud Radio 

Le tableau de bord politique  Février 2021 

 

 

 

10 




In [4]:
with open(file_path,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        page_content=pdf_reader.getPage(11).extractText()
# print(get_base(file_path))
(page_content)


' \nIfop\n-\nFiducial\n \npour Paris Match\n \net Sud Radio\n \nLe tableau de bord politique\n \n\n \nFévrier 2021\n \n10\n \n\np\nrésident\n \nde la République\n \n \n \n \n \n \n \nQuestion\n \n:\n \nPour chacune des\n \nappréciations suivantes, dites\n \nsi elle correspond bien \n\np\nrésident de la République\n.\n \n \n \n \n \n \n \n \n \n \n \n\n \nCorrespond très bien \n \n \n \n\n \nCorrespond très mal\n \n\n \nNe se prononce pas\n \nTOTAL \nCorres\n-\npond \nbien\n \nTOTAL \nCorres\n-\npond mal\n \n \n \n \n5\n3\n \n%\n \n4\n7\n \n%\n \n4\n1\n \n%\n \n5\n9\n \n%\n \n40\n \n%\n \n6\n0\n \n%\n \n38\n \n%\n \n62\n \n%\n \n33\n \n%\n \n67\n \n%\n \n \n \n \n \n \n \n12%\n9%\n6%\n9%\n5%\n41%\n32%\n34%\n29%\n28%\n27%\n32%\n34%\n35%\n34%\n20%\n27%\n26%\n27%\n33%\nDéfend bien les intérêts \n\nRenouvelle la fonction\nprésidentielle\nMène une bonne\npolitique économique\n\ndes Français \nEst proche des\npréoccupations des\nFrançais\n'

In [5]:
def pretraitement(text):
    text=text.replace('opinion\n \n»','opinion\n \n')
    return text

In [6]:
def get_names(file_path,page):
    with open(file_path,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        page_content=pdf_reader.getPage(page).extractText()
    page_content=pretraitement(page_content)
    text=page_content[page_content.lower().find('bonne opinion'):]
    text=text.replace('\n \n','\n\n')
    text=text.split('\n\n')[1:]
    noms=[]
    for i in range(text.index('ENSEMBLE')):
        nom=text[i]
        nom=nom.replace('\n','')
        print(nom)
        fullname=nom.split(' ')
        fullname[1]=''.join([x.upper() for x in fullname[1:]])
        nom=' '.join(fullname[:2])
        noms.append(nom)
    return noms
# get_names(file_path,11)

ValueError: 'ENSEMBLE' is not in list

In [24]:
def get_pos_opinions(file_path,page,nb_pers):
    with open(file_path,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        page_content=pdf_reader.getPage(page).extractText()
    page_content=pretraitement(page_content)
    text=page_content[page_content.lower().find('bonne opinion'):]
    opinions=[int(x) for x in re.findall('\d+',text)[:nb_pers]]
    return opinions
get_pos_opinions(file_path,11,8)
    

[69, 67, 62, 52, 52, 49, 49, 48]

In [25]:
def get_base(file_path,page=3):
    text = extract_text(file_path,page_numbers=[page])
    txt_idx="auprès d’un échantillon de "
    idx=text.find(txt_idx)+len(txt_idx)
    return float(re.findall('[0-9]+',text[idx:idx+10].replace(' ',''))[0])

In [26]:
def extract_ifop_opinions(file_path,p_start,p_end):
    base=get_base(file_path)
    date=file_path.split("/")[-1].rstrip('.pdf')
    columns=['Nom','Base','Date','Opinion positive']
    df=pd.DataFrame(columns=columns)
    for p_nb in range(p_start,p_end+1):
        noms=get_names(file_path,p_nb)
        print(noms)
        opinions=get_pos_opinions(file_path,p_nb,len(noms))
        c_df=pd.DataFrame(columns=columns)
        c_df['Nom']=noms
        c_df['Opinion positive']=opinions
        c_df['Date']=date
        c_df['Base']=base
        df=pd.concat([df,c_df],ignore_index=True)
    return df

In [28]:
filename="2021-02-05"
file_path='rapport_personnalite/'+filename+'.pdf'
p_start,p_end=11,17
# Enlever 1 par rapport au pdf aux pages
df=extract_ifop_opinions(file_path,p_start,p_end)
df.to_pickle('data_perso/'+filename+'.p')
df

Nicolas Hulot
Edouard Philippe
Roselyne Bachelot
Bruno Le Maire
Martine Aubry
Emmanuel Macron
Nicolas Sarkozy
Bernard Cazeneuve
['Nicolas HULOT', 'Edouard PHILIPPE', 'Roselyne BACHELOT', 'Bruno LEMAIRE', 'Martine AUBRY', 'Emmanuel MACRON', 'Nicolas SARKOZY', 'Bernard CAZENEUVE']
Jean-Yves Le Drian
Anne Hidalgo
Olivier Véran
François Baroin
François Bayrou
Jean Castex
Xavier Bertrand
François Hollande
['Jean-Yves LEDRIAN', 'Anne HIDALGO', 'Olivier VÉRAN', 'François BAROIN', 'François BAYROU', 'Jean CASTEX', 'Xavier BERTRAND', 'François HOLLANDE']
Ségolène Royal
Rachida Dati
Valérie Pécresse
Eric Dupond-Moretti
Laurent Wauquiez
Gérard Larcher
Jean-Michel Blanquer
Christian Estrosi
['Ségolène ROYAL', 'Rachida DATI', 'Valérie PÉCRESSE', 'Eric DUPOND-MORETTI', 'Laurent WAUQUIEZ', 'Gérard LARCHER', 'Jean-Michel BLANQUER', 'Christian ESTROSI']
Gérald Darmanin
Jean-Luc Mélenchon
Nicolas Dupont-Aignan
François Ruffin
Hervé Morin
Christophe Castaner
Marlène Schiappa
Marine Le Pen
['Gérald DARMAN

,Nom,Base,Date,Opinion positive
0,Nicolas HULOT,1005.0,2021-01-14,69
1,Edouard PHILIPPE,1005.0,2021-01-14,67
2,Roselyne BACHELOT,1005.0,2021-01-14,62
3,Bruno LEMAIRE,1005.0,2021-01-14,52
4,Martine AUBRY,1005.0,2021-01-14,52
5,Emmanuel MACRON,1005.0,2021-01-14,49
6,Nicolas SARKOZY,1005.0,2021-01-14,49
7,Bernard CAZENEUVE,1005.0,2021-01-14,48
8,Jean-Yves LEDRIAN,1005.0,2021-01-14,47
9,Anne HIDALGO,1005.0,2021-01-14,46


In [29]:
# import datetime
# df=pd.read_pickle('data/2021-01-21.p')
# df.loc[0,df.columns[4:13]]=[7,34,41,34,25,59,0,-4,+4]
# df.loc[1,df.columns[4:13]]=[4,34,38,34,28,62,0,-3,+3]
# df.Base=1004
# df.Date=datetime.date(2021,2,5)

In [32]:
# df.to_pickle('data/2021-02-05.p')

In [29]:
# df.to_pickle('data/'+filename+'.p')

In [ ]:
# df3=pd.concat([df,df2],ignore_index=True)
# df3.to_pickle('data/'+filename+'.p')

In [ ]:
# Reading IPSOS report

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]